# Dataset

In [ ]:
import pandas as pd
df = pd.read_csv("burmese_nli_dataset.csv")

In [ ]:
df.head()

,premise,hypothesis,genre,label
0,လူငယ်တွေသဘာဝပတ်ဝန်းကျင်ကိုချစ်မြတ်နိုးလာအောင်သ...,ကျောင်းသားတွေကိုသဘာဝပတ်ဝန်းကျင်အကြောင်းသင်ပေးန...,travel,0
1,လူငယ်တွေသဘာဝပတ်ဝန်းကျင်ကိုချစ်မြတ်နိုးလာအောင်သ...,ကျွန်းပင်နဲ့ပိတောက်ပင်တွေကိုအဓိကစိုက်တာပါ,travel,1
2,လူငယ်တွေသဘာဝပတ်ဝန်းကျင်ကိုချစ်မြတ်နိုးလာအောင်သ...,သစ်ပင်တွေအားလုံးကိုခုတ်လှဲပစ်နေတာပါ,travel,2
3,ဟုတ်တယ်အခုခေတ်လူငယ်တွေကဘာသာစကားအသစ်တွေကိုသင်ယူ...,လူငယ်တွေကသင်ယူနိုင်စွမ်းမြင့်မားကြတယ်,telephone,0
4,ဟုတ်တယ်အခုခေတ်လူငယ်တွေကဘာသာစကားအသစ်တွေကိုသင်ယူ...,အများစုကဂျပန်စာနဲ့ကိုရီးယားစာကိုသင်နေကြတာပါ,telephone,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10443 entries, 0 to 10442
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     10443 non-null  object
 1   hypothesis  10443 non-null  object
 2   genre       10443 non-null  object
 3   label       10443 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 326.5+ KB


In [ ]:
df["label"].value_counts()

,count
label,
0,3605
1,3462
2,3363


In [ ]:
df["hypothesis"].duplicated().sum()

np.int64(0)

# Library Installation

In [ ]:
!pip install burmese_tools

In [ ]:
!pip install myanmartools

In [ ]:
!pip install -q transformers datasets accelerate scikit-learn


# Preprocessing

In [ ]:
from burmese_tools import tools
from myanmartools import ZawgyiDetector
import re
import unicodedata

detector = ZawgyiDetector()

def normalize_myanmar(text):
    # Unicode normalize
    text = unicodedata.normalize("NFC", text)
    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

def convert(text):
    score = detector.get_zawgyi_probability(text)
    if score > 0.95:
        text = tools.zaw2uni(text)
    text = normalize_myanmar(text)
    return text


# Importing Library

In [ ]:
import random
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score

In [ ]:
df["premise"] = df["premise"].apply(lambda x: normalize_myanmar(convert(x)))
df["hypothesis"] = df["hypothesis"].apply(lambda x: normalize_myanmar(convert(x)))


# Set Seed

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = True


set_seed(42)


# Split Train/Test/Val - 70-15-15

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(
    n_splits=1,
    test_size=0.3,
    random_state=42
)

train_idx, temp_idx = next(splitter.split(df, groups=df["premise"]))

train_df = df.iloc[train_idx]
temp_df  = df.iloc[temp_idx]

splitter2 = GroupShuffleSplit(
    n_splits=1,
    test_size=0.5,
    random_state=42
)

val_idx, test_idx = next(splitter2.split(temp_df, groups=temp_df["premise"]))

val_df  = temp_df.iloc[val_idx]
test_df = temp_df.iloc[test_idx]

In [ ]:
len(test_df)

1575

In [ ]:
len(train_df)

7347

In [ ]:
train_premises = set(train_df["premise"])
test_premises  = set(test_df["premise"])
val_premises = set(val_df["premise"])
overlap = train_premises & test_premises
overlap2 = train_premises & val_premises
print("Premise overlap train and test", len(overlap))
print("Premise overlap of train and val",len(overlap2))

Premise overlap train and test 0
Premise overlap of train and val 0


In [ ]:
train_hypo = set(train_df["hypothesis"])
test_hypo  = set(test_df["hypothesis"])
val_hypo  = set(val_df["hypothesis"])
overlap = train_hypo & test_hypo
overlap2 = train_hypo & val_hypo
print("Hypothesis overlap of train and test:", len(overlap))
print("Hypothesis Overlap of train and val", len(overlap2))


Hypothesis overlap of train and test: 0
Hypothesis Overlap of train and val 0


# Tokenizer

## Load Tokenizer

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

## Tokenization Function

In [ ]:
def tokenize_fn(batch):
    tokenized = tokenizer(
        batch["premise"],
        batch["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized["labels"] = batch["label"]
    return tokenized


# Convert to Huggingface Dataset

In [ ]:
train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(tokenize_fn, batched=True, remove_columns=val_ds.column_names)
test_ds  = test_ds.map(tokenize_fn, batched=True, remove_columns=test_ds.column_names)
cols = ["input_ids", "attention_mask", "labels"]

train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)


Map:   0%|          | 0/7290 [00:00<?, ? examples/s]

Map:   0%|          | 0/1576 [00:00<?, ? examples/s]

Map:   0%|          | 0/1564 [00:00<?, ? examples/s]

In [ ]:
print(train_ds[1])


{'input_ids': tensor([     0,      6,   9547,  50739,   1364,  76313,   1489,  56274,  23109,
         19589,  12104, 126323, 230156,   7457,  50001,  50001,  78344,  78344,
         50993,  17096, 188436,   1364,  34506, 170376, 172363,  86717,  83470,
         22253,  19317,  23376,  95224,  66018,  17096,  38258,      2,      6,
        153404,  41845,  39503, 203121,   1489, 233912, 154421,  13172, 214746,
         38258,      2,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,   

# Load the model

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)



model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: xlm-roberta-base
Key                         | Status     | 
----------------------------+------------+-
lm_head.layer_norm.bias     | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
classifier.dense.bias       | MISSING    | 
classifier.dense.weight     | MISSING    | 
classifier.out_proj.bias    | MISSING    | 
classifier.out_proj.weight  | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


## Metrics (ACC+F1 score)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),

    }


# Training Args and Training Process

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./xlm_roberta_nli_v2",
    eval_strategy="epoch",
    save_strategy="epoch",

    learning_rate=1.5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps = 1,
    num_train_epochs=10,

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    weight_decay=0.02,
    warmup_ratio=0.1,
    logging_steps=100,

    fp16=True,
    seed=42,
    report_to="none",

    save_total_limit=2,
)


warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,

    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=1
        )
    ]
)



## Trainer

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.757902,0.791948,0.641497,0.613437
2,0.676441,0.620695,0.729061,0.726486
3,0.580144,0.686108,0.744289,0.746157
4,0.455421,0.641453,0.748096,0.748830
5,0.384802,0.643417,0.764594,0.764614
6,0.356386,0.729629,0.760787,0.760709


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

TrainOutput(global_step=2736, training_loss=0.5467407201465807, metrics={'train_runtime': 773.8429, 'train_samples_per_second': 94.205, 'train_steps_per_second': 5.893, 'total_flos': 2877145222855680.0, 'train_loss': 0.5467407201465807, 'epoch': 6.0})

# Test Data Accuracy and Metrics

In [ ]:
test_metrics = trainer.evaluate(test_ds)
print(test_metrics)



{'eval_loss': 0.5780307650566101, 'eval_accuracy': 0.7877237851662404, 'eval_f1': 0.7875596745154428, 'eval_runtime': 3.72, 'eval_samples_per_second': 420.434, 'eval_steps_per_second': 26.344, 'epoch': 6.0}


In [ ]:
# Save the result
import json
with open("test_metrics_v3.json", "w") as f:
    json.dump(test_metrics, f, indent=4)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(all_labels,all_preds)
print(cm)


[[481  50  18]
 [ 78 418  20]
 [ 14  20 476]]


## Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(
    classification_report(
        all_labels,
        all_preds,
        target_names=["entailment", "neutral", "contradiction"]  # adjust order!
    )
)


               precision    recall  f1-score   support

   entailment       0.84      0.88      0.86       549
      neutral       0.86      0.81      0.83       516
contradiction       0.93      0.93      0.93       510

     accuracy                           0.87      1575
    macro avg       0.87      0.87      0.87      1575
 weighted avg       0.87      0.87      0.87      1575



# Upload to Hugginface

In [ ]:
!pip install -U transformers datasets huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.4.0
    Uninstalling huggingface_hub-1.4.0:
      Successfully uninstalled huggingface_hub-1.4.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0

In [ ]:
from huggingface_hub import login
login()


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer




In [ ]:
model     # trained model

XLMRobertaForSequenceClassification(
  (classifier): XLMRobertaClassificationHead(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=3, bias=True)
  )
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Li

In [ ]:
import torch

In [ ]:
print(model.config.num_labels)


3


In [ ]:
repo_id = "emilyyy04/xlm-roberta-base-burmese-nli-v3"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...j1ti7y7/model.safetensors:   0%|          | 13.5kB / 1.11GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpfnme6ced/tokenizer.json:  50%|####9     | 8.34MB / 16.8MB            

CommitInfo(commit_url='https://huggingface.co/emilyyy04/xlm-roberta-base-burmese-nli-v3/commit/77aaaa4814d8df6d0031ad73cd14237bd143b2b0', commit_message='Upload tokenizer', commit_description='', oid='77aaaa4814d8df6d0031ad73cd14237bd143b2b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/emilyyy04/xlm-roberta-base-burmese-nli-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='emilyyy04/xlm-roberta-base-burmese-nli-v3'), pr_revision=None, pr_num=None)

# Call Infernce from Hugging face

In [ ]:
MODEL_ID = "emilyyy04/xlm-roberta-base-burmese-nli-v3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [ ]:
train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(tokenize_fn, batched=True, remove_columns=val_ds.column_names)
test_ds  = test_ds.map(tokenize_fn, batched=True, remove_columns=test_ds.column_names)
cols = ["input_ids", "attention_mask", "labels"]

train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)


Map:   0%|          | 0/7347 [00:00<?, ? examples/s]

Map:   0%|          | 0/1521 [00:00<?, ? examples/s]

Map:   0%|          | 0/1575 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)
model.eval()

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification(
  (classifier): XLMRobertaClassificationHead(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=3, bias=True)
  )
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Li

In [ ]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

In [ ]:
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

In [ ]:
import torch.nn.functional as F
@torch.no_grad()
def nli_inference(premise: str, hypothesis: str, max_length: int = 256):
    premise = convert(premise)
    hypothesis= convert(hypothesis)
    inputs = tokenizer(
        premise,
        hypothesis,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(DEVICE)

    outputs = model(**inputs)
    logits = outputs.logits

    probs = F.softmax(logits, dim=-1)[0]
    pred_id = torch.argmax(probs).item()

    return {
        "label": label_map[pred_id],
        "confidence": round(probs[pred_id].item(), 4),
        "scores": {
            label_map[i]: round(probs[i].item(), 4)
            for i in range(len(label_map))
        }
    }

In [ ]:
premise = "ကျောင်းသားလေးဟာ တက္ကသိုလ်စာကြည့်တိုက်ထဲမှာ စာအုပ်ပုံကြီးကြား နစ်ပြီး အိပ်ပျော်သွားခဲ့တယ်။"
hypothesis = "စာကြည့်တိုက်ထဲမှာ စာဖတ်နေတဲ့ ကျောင်းသားလေး အိပ်ပျော်သွားတယ်၊"


In [ ]:
premise = "ကျောင်းသားလေးဟာ တက္ကသိုလ်စာကြည့်တိုက်ထဲမှာ စာအုပ်ပုံကြီးကြား နစ်ပြီး အိပ်ပျော်သွားခဲ့တယ်။"
hypothesis = "သူဟာ နောက်နေ့မနက် စာမေးပွဲအတွက် ပြင်ဆင်နေတာဖြစ်တယ်"


In [ ]:

premise = "ကျောင်းသားလေးဟာ တက္ကသိုလ်စာကြည့်တိုက်ထဲမှာ စာအုပ်ပုံကြီးကြား နစ်ပြီး အိပ်ပျော်သွားခဲ့တယ်။"
hypothesis = "ကျောင်းသားလေးဟာ စာကြည့်တိုက်ထဲမှာ တစ်ညလုံး လမ်းလျှောက်နေတယ်"


In [ ]:
premise = "ပြည်နယ်များ၏ ထိန်းချုပ်မှုကို လေးစားလိုက်ခြင်းပင် ဖြစ်သည်။"
hypothesis = "ပြည်နယ်များကို ခွင့်မပြုထားသောကြောင့် သူတို့သည် သူတို့၏ ရွေးကောက်ပွဲ လုပ်ငန်းစဉ်ကို ဘယ်တော့မှ မပြောင်းလဲကြပါ။"


In [ ]:
premise = "မိုးရွာနေသည်။"
hypothesis = "လမ်းများ စိုနေသည်။"

In [ ]:
premise ="ကလေးလေးပန်းခြံထဲမှာ ဘောလုံးကစားနေတယ်။"
hypothesis = "ကလေးတစ်ယောက်အပြင်မှာ ရှိနေတယ်။"


In [ ]:
premise = "မောင်မောင်က ပန်းသီးကို ဓားနဲ့ လှီးနေတယ်။"
hypothesis = "မောင်မောင်မှာ သစ်သီးရှိတယ်။"

In [ ]:
premise ="မိုးရွာနေသောကြောင့် ပြိုင်ပွဲကို ရပ်နားခဲ့သည်။"
hypothesis ="မိုးမရွာသောကြောင့် ပြိုင်ပွဲကို ဆက်လုပ်ခဲ့သည်။"


In [ ]:
premise ="ကလေးများသည် စာကြည့်တိုက်တွင် စာအုပ်ဖတ်နေကြသည်။"
hypothesis ="ကလေးများသည် စာဖတ်နေကြသည်။"

In [ ]:
premise = "သူက ရန်ကုန်မှာ နေထိုင်ပါတယ်။"
hypothesis = "သူက မြန်မာနိုင်ငံမှာ နေထိုင်ပါတယ်။"


In [ ]:
premise = "သူမက ဆရာမတစ်ယောက် ဖြစ်ပါတယ်။"
hypothesis = "သူမက ကျောင်းမှာ သင်ကြားနေပါတယ်။"


In [ ]:
premise = "သူက စာဖတ်နေပါတယ်။"
hypothesis = "သူက စာမေးပွဲအတွက် ပြင်ဆင်နေပါတယ်။"


In [ ]:
premise = "သူမက မနက်ခင်းမှာ လေ့ကျင့်ခန်းလုပ်တယ်။"
hypothesis = "သူမက အားကစားပြိုင်ပွဲမှာ ဝင်ပြိုင်မယ်။"


In [ ]:
premise = "သူက ကော်ဖီ သောက်နေတယ်။"
hypothesis = "သူက အလုပ်မှာရှိနေတယ်။"


In [ ]:
premise = "သူက ကျောင်းသား ဖြစ်ပါတယ်။"
hypothesis = "သူက ကျောင်းဆရာ ဖြစ်ပါတယ်။"


In [ ]:
premise = "ဒီနေ့ မိုးမရွာပါဘူး။"
hypothesis = "ဒီနေ့ မိုးရွာနေပါတယ်။"


In [ ]:
premise = "သူမက အိမ်မှာရှိနေပါတယ်။"
hypothesis = "သူမက အပြင်ထွက်သွားပါတယ်။"


In [ ]:
premise = "သူက မော်တော်ကား မောင်းသွားခဲ့တယ်။"
hypothesis = "သူက လမ်းပေါ်မှာရှိခဲ့တယ်။"


In [ ]:
premise="နေမကောင်းတာမဟုတ်ဘူး"
hypothesis = "နေကောင်းတယ်"

In [ ]:
premise ="နွေဦးပေါက်ချိန်တွင်ပလောင်တိုင်းရင်းသားတို့သည်လက်ဖက်စိမ်းများကိုခူးဆွတ်ပြီးဈေးကွက်သို့တင်ပို့ရန်ပြင်ဆင်ကြသည်"
hypothesis = "ပလောင်ဒေသတွင်လက်ဖက်ပင်များလုံးဝမစိုက်ပျိုးကြပါ"

In [ ]:
premise = "ဒီဗျူဟာကိုကျွန်တော်အထူးသတိမပြုခဲ့ပါဘူး"
hypothesis = "ကျွန်တော်သည်ဤဗျူဟာကိုအသေးစိတ်ကျွမ်းကျင်စွာသိရှိပါသည်"

In [ ]:
premise = "သစ်တောပြုန်းတီးမှုကရာသီဥတုပြောင်းလဲမှုကိုပိုပြီးဆိုးရွားစေပါတယ်"
hypothesis ="သစ်ပင်တွေစိုက်တာကရာသီဥတုကိုပျက်စီးစေတယ်"

In [ ]:
premise = "အခုတလောရန်ကုန်မြို့ရဲ့လမ်းမတွေပေါ်မှာကားပိတ်တာကတကယ်ကိုသည်းခံရခက်တဲ့အခြေအနေကိုရောက်နေပြီးခရီးတစ်ခုသွားဖို့အချိန်အများကြီးပေးနေရတယ်"
hypothesis = "ရန်ကုန်မှာသွားလာရေးကအရင်ကထက်အများကြီးပိုခက်ခဲလာတယ်"

In [ ]:
premise = "ဟုတ်တယ်ဗျခင်ဗျားပြောတာမှန်တယ်အခုခေတ်မှာလူငယ်တွေကနည်းပညာကိုအရင်ကထက်ပိုပြီးကျွမ်းကျင်လာကြတော့အလုပ်အကိုင်အခွင့်အလမ်းသစ်တွေအများကြီးပေါ်လာတာပေါ့"
hypothesis = "နည်းပညာတိုးတက်လာတာကလူငယ်တွေအတွက်အခွင့်အလမ်းကောင်းတွေဖြစ်စေတယ်"

In [ ]:
premise = "သူကစားပွဲပေါ်မှာပြန့်ကျဲနေတဲ့စာရွက်စာတမ်းတွေကိုအမြန်ဆုံးသိမ်းဆည်းလိုက်ပြီးဘယ်သူမှမမြင်ခင်အခန်းထဲကနေခိုးကြောင်ခိုးဝှက်ထွက်သွားတယ်"
hypothesis ="သူတစ်ခုခုကိုဖုံးကွယ်ချင်နေတာကြောင့်အလျင်စလိုလုပ်နေတာပါ"

In [ ]:
premise = "အဲဒါကနှေးတယ်ဗျအခုဈေးကွက်ထဲမှာအဲဒီထက်အများကြီးပိုကောင်းတဲ့စက်တွေအများကြီးရှိပါတယ်"
hypothesis ="ဒါကအမြန်ဆုံးစက်ပါပဲဒီထက်ကောင်းတာကိုရှာတွေ့မှာမဟုတ်တော့ဘူး"

In [ ]:
premise ="ယခုနှစ်ပွဲတော်ရာသီတွင်မွန်ပြည်နယ်ရှိကျိုက်ထီးရိုးဘုရားသို့ဘုရားဖူးသွားရန်အတွက်ကားလက်မှတ်များအားလုံးရောင်းကုန်သွားပြီဖြစ်သည်"
hypothesis ="ခရီးသွားအများအပြားသည်ကျိုက်ထီးရိုးသို့သွားရန်ပြင်ဆင်နေကြသည်"

In [ ]:
premise ="မနက်အစောပိုင်းအချိန်တွင်ခရီးသည်များသည်နယ်ဝေးကားဂိတ်၌လက်မှတ်ဝယ်ယူရန်တန်းစီစောင့်ဆိုင်းနေကြသည်"
hypothesis="ကားလက်မှတ်စျေးနှုန်းများမှာယခင်ကထက်နှစ်ဆမြင့်တက်နေသည်"


In [ ]:
result = nli_inference(premise,hypothesis)
result

{'label': 'contradiction',
 'confidence': 0.9957,
 'scores': {'entailment': 0.0019, 'neutral': 0.0023, 'contradiction': 0.9957}}

In [ ]:
import torch.nn.functional as F

def predict_nli_pipeline(premise, hypothesis):
    # Preprocess
    premise = convert(premise)
    hypothesis = convert(hypothesis)

    # Tokenize & inference
    inputs = tokenizer(
        premise,
        hypothesis,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(DEVICE)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)[0]  # shape = [3]

    # Predicted class
    pred_class = torch.argmax(probs).item()

    # Return pred + confidence
    return pred_class, {label_map[i]: round(float(probs[i]), 3) for i in range(3)}



In [ ]:

pred_class, conf_scores = predict_nli_pipeline(premise, hypothesis)

print("Predicted label:", label_map[pred_class])
print("Confidence per class:", conf_scores)


Predicted label: contradiction
Confidence per class: {'entailment': 0.183, 'neutral': 0.101, 'contradiction': 0.716}
